In [1]:
from Rings import Zomega, Ztau, Exact_U
from norm_equation import easy_factor, easy_solveable, solve_norm_equation
from exact_synthesize import exact_synthesize, exact_synthesize_weave
from Circuits import FTCircuit, Braid
from mpmath import mp, ceil, floor
import cmath
import math
import random
import numpy as np
from toqito.random import random_unitary

In [2]:
mp.dps = 50

`approx_real` approximates a real number x to precision \(\tau^{n-1}(1 - \tau^n)\)

In [3]:
FibArray = [1, 1]
 
def Fib(n):
    if n<0:
        print("Incorrect input")
    elif n<= len(FibArray):
        return FibArray[n-1]
    else:
        temp_fib = Fib(n-1)+Fib(n-2)
        FibArray.append(temp_fib)
        return temp_fib    

In [4]:
def approx_real(x,n):
    p, q = Fib(n), Fib(n+1)
    u, v = (-1)**(n+1)*Fib(n), (-1)**n * Fib(n-1)
    c = round(x*q)
    a = c*v + p * round(c*u/q)
    b = c*u - q * round(c*u/q)

    return Ztau(a, b)

In [5]:
print(approx_real(mp.pi, 20))

(-317 + 518τ)


`random_sample` generates an element of Z[omega] approximating (r*phi^m) * exp(-i theta/2) to within precision epsilon

In [6]:
phi = (mp.sqrt(mp.mpf(5)) + 1)/2
tau = (mp.sqrt(mp.mpf(5)) - 1)/2

In [7]:
def Rz(angle):
    return np.array([
        [cmath.exp(-0.5j*angle), 0],
        [0, cmath.exp(0.5j*angle)]
    ])

def Pauli_X():
    return np.array([
        [0, 1],
        [1, 0]
    ])

def F():
    return np.array([
        [tau, sqrt(tau)],
        [sqrt(tau), -1*tau]
    ])

In [8]:
test = mp.mpf(3)
print(int(float(mp.nstr(test,1))))

3


In [9]:
def random_sample(theta, epsilon, r):
    C = mp.sqrt(phi/(4*r))
    m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon*r, tau)), 1))) + 1
    N = int(float(mp.nstr(mp.ceil(phi**m), 1)))

    y_min = r*phi**m *(mp.sin(theta) - epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) + epsilon*mp.sin(theta))/2)
    y_max = r*phi**m *(mp.sin(theta) + epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) - epsilon*mp.sin(theta))/2)
    x_max = r*phi**m *((1 - epsilon**2/2)*mp.cos(theta) - epsilon*mp.sqrt(1 - epsilon**2/4)*mp.sin(theta))
    x_c = x_max - r*epsilon**2*phi**m/(4*mp.cos(theta))

    j = random.randint(1, N-1)
    y = y_min + j*(y_max - y_min)/N
    a_y = approx_real(y/mp.sqrt(2-tau), m)
    x = x_c - (a_y.value()*mp.sqrt(2-tau) - y_min)*mp.tan(theta)
    a_x = approx_real(x,m)

    return Zomega.fromTau(a_x) + Zomega(-1,2,-1,1)*Zomega.fromTau(a_y)


In [10]:
r = 1
epsilon = 10**(-2)
sample = random_sample(0, epsilon, r)
print("sample: " + str(sample))

sample: (182 + -34ω + 83ω² + -61ω³)


In [11]:
def compile_R_rotation(angle, epsilon):
    epsilon = epsilon * epsilon
    C = mp.sqrt(phi/4)
    m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon, tau)), 1))) + 1
    theta = 0
    for k in range(10):       
        adjusted_angle = (-angle/2 - mp.pi*k/5) - floor((-angle/2 - mp.pi*k/5)/(2*mp.pi)) * (2*mp.pi)
        if adjusted_angle >= 0 and adjusted_angle <= mp.pi/5:
            theta = adjusted_angle
            break
    
    not_found, u, v = True, 0, 0

    while not_found:
        u0 = random_sample(theta, epsilon, 1)
        phi_t = Ztau(1, 1)
        xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
        factors = easy_factor(xi)
        if easy_solveable(factors):
            not_found = False
            #print("found one!")
            #print("u0 angle: " + str(cmath.phase(u0.value())))
            u = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
            #print("norm solution: " + str(solve_norm_equation(xi)))
            v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
    
    print("u, v = " + str(u) + ", " + str(v))
    #print("u,v value: " + str(u.value()) + ", " + str(v.value()))
    #print("u angle: " + str(cmath.phase(u.value())))
    print("distance = " + str(mp.sqrt(1 - 0.5 * abs((u.value()*mp.exp(mp.mpc(1j)*angle/2) + (u.star().value()*mp.exp(mp.mpc(-1j)*angle/2)))))))
    #print("-----------------------")
    #print(Exact_U(u,v,5).value())
    Circuit = exact_synthesize(Exact_U(u, v, 5))

    return Circuit

In [30]:
# Testing compile_R_rotation

angle = 0.3
print(Rz(angle))
FT = compile_R_rotation(angle, 0.0001)
print(FT.toBraid())
print(FT.toBraid().numerical_approximate())


[[0.98877108-0.14943813j 0.        +0.j        ]
 [0.        +0.j         0.98877108+0.14943813j]]
u, v = (-191550867544865616 + 427599163406597260ω + -381934165690772948ω² + 117663349144475568ω³), (-4861476148834106 + 205915207349250085ω + -325311770647258251ω² + 198049173704939559ω³)
distance = 0.000090402335424546331245489770946120417661043796746481


KeyboardInterrupt: 

In [38]:
u = Zomega(-255009288268180616, 341634187605978540, -140162031400595300, -70979508258857508)
v = Zomega(45639281038229241, -245501868762956335, 323384460018115951, -171655960820992834)
print("u,v value: " + str(u.value()) + ", " + str(v.value()))
print(u.N())


u,v value: (36+8j), (40+32j)
294405716540324035696780176464778496


In [21]:
def compile_RX_rotation(angle, epsilon):
    epsilon = epsilon * epsilon
    r = sqrt(phi)
    C = sqrt(phi/(4*r))
    m = ceil(log(C*epsilon*r, tau)) + 1
    theta = 0
    for k in range(10):
        adjusted_angle = (angle/2 + pi/2 - pi*k/5) - ((angle/2 + pi/2 - pi*k/5)//(2*pi)) * (2*pi)
        if adjusted_angle >= 0 and adjusted_angle <= pi/5:
            theta = adjusted_angle
            break
    
    not_found, u, v = True, 0, 0

    while not_found:
        u0 = random_sample(theta, epsilon, r)
        tau_t = Ztau(0,1)
        phi_t = Ztau(1,1)
        xi = phi_t**(2*m) - tau_t * Ztau.fromOmega(u0*u0.star())
        factors = easy_factor(xi)
        if easy_solveable(factors):
            not_found = False
            #print("found one!")
            #print(k)
            v = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
            #print("norm solution: " + str(solve_norm_equation(xi)))
            u = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
    
    # v = Zomega(-1,0,0,0) * v.star()
    ex = Exact_U(u, v, 5)
    print(ex.value())
    #print("-----------------------")
    Circuit = exact_synthesize(ex)

    return Circuit

In [ ]:
# Testing compile_RX_rotation

angle = pi/2
print(np.matmul(Rz(angle),Pauli_X()))
FT = compile_RX_rotation(angle, 0.0001)
print(FT)
print(FT.numerical_approximate())
print(FT.toBraid())
print(FT.toBraid().numerical_approximate())

In [26]:
def decompose_unitary(U):
    # decompose U into e^i*delta Rz(alpha)*F*Rz(beta)*F*Rz(gamma) or e^id Rz(a)*F*Rz(b)*F*Rz(c) X
    X = False
    a, b, c, d = U[0,0], U[0,1], U[1,0], U[1,1]
    if abs(a) < 0.5:
        X = True
        a, b, c, d = U[0,1], U[0,0], U[1,1], U[1,0]
    
    delta = 0.5 * cmath.phase(a*d - b*c)

    a, b, c, d = cmath.exp(-1j * delta)*a, cmath.exp(-1j * delta)*b, cmath.exp(-1j * delta)*c, cmath.exp(-1j * delta)*d, 
    
    beta = acos(1 - abs(c)**2/(2*tau**3))
    alpha = -(cmath.phase(a) - cmath.phase(cmath.exp(1j*beta) + tau)
              - cmath.phase(c) + cmath.phase(1 - cmath.exp(1j*beta)))
    gamma = -2*(cmath.phase(a) - cmath.phase(cmath.exp(1j*beta) + tau)) - alpha - beta
    
    return (alpha, beta, gamma, delta, X)


In [27]:
def compile_unitary(U, epsilon):
    # break U into rotations and then compile them individually
    # has precision epsilon^(1/6)
    (alpha, beta, gamma, delta, X) = decompose_unitary(U)
    #print(cmath.exp(1j*delta)*np.matmul(Rz(alpha), np.matmul(F(), np.matmul(Rz(beta), np.matmul(F(), Rz(gamma))))))
    A = compile_R_rotation(alpha, epsilon)
    B = compile_R_rotation(beta, epsilon)
    if X:
        C = compile_RX_rotation(gamma, epsilon)
    else:
        C = compile_R_rotation(gamma, epsilon)


    # debug block
    A_num, B_num, C_num = A.numerical_approximate(), B.numerical_approximate(), C.numerical_approximate()
    estimate = cmath.exp(delta*pi*1j) * np.matmul(A_num, np.matmul(F(), np.matmul(B_num, np.matmul(F(), C_num))))
    print(abs(estimate[0]))

    F_circ = FTCircuit()
    F_circ.join("F", 1)

    A_braid, B_braid, C_braid, F_braid = A.toBraid(), B.toBraid(), C.toBraid(), F_circ.toBraid()

    total_braid = A_braid + F_braid + B_braid + F_braid + C_braid
    total_braid.addPhase(delta)

    return total_braid

In [28]:
'''
U = random_unitary(2)
print(U)
print(decompose_unitary(U))
braid = compile_unitary(U, 0.0001)
print(braid)
print(braid.numerical_approximate())
'''

'\nU = random_unitary(2)\nprint(U)\nprint(decompose_unitary(U))\nbraid = compile_unitary(U, 0.0001)\nprint(braid)\nprint(braid.numerical_approximate())\n'

In [25]:
def compile_R_weave(angle, epsilon):
    weave = False
    while not weave:
        C = mp.sqrt(phi/4)
        m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon, tau)), 1))) + 1
        theta = 0
        for k in range(10):       
            adjusted_angle = (-angle/2 - mp.pi*k/5) - floor(((-angle/2 - mp.pi*k/5)/(2*mp.pi))) * (2*mp.pi)
            if adjusted_angle >= 0 and adjusted_angle <= mp.pi/5:
                theta = adjusted_angle
                break
        
        not_found, u, v = True, 0, 0

        while not_found:
            u0 = random_sample(theta, epsilon, 1)
            phi_t = Ztau(1, 1)
            xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
            #print("xi = " + str(xi))
            factors = easy_factor(xi)
            #for factor in factors:
            #    print(factor[0])
            if easy_solveable(factors):
                not_found = False
                u = Zomega(0,1,0,0)**k * Zomega.fromTau(Ztau(0,1))**m * u0
                v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
        
        print("u, v = " + str(u) + ", " + str(v))
        # print("distance = " + str(mp.sqrt(1 - 0.5 * abs((u.value()*mp.exp(mp.mpc(1j)*angle/2) + (u.star().value()*mp.exp(mp.mpc(-1j)*angle/2)))))))   
        (Circuit, Remainder) = exact_synthesize_weave(Exact_U(u, v, 5))
        # print(Circuit)
        # print(Remainder)

        if len(Remainder.circuit) == 0:
            weave = True

    return Circuit

In [59]:
angle = 2*mp.pi
print(Rz(angle))
is_desired_phase = False
while not is_desired_phase:
    FT = compile_R_weave(angle, 0.000000037)
    print(FT)
    weave = FT.toBraid()
    print(weave)
    print(weave.numerical_approximate())
    print(weave.individual_winding())
    if weave.phase == 0 and weave.individual_winding() == (0,0):
        is_desired_phase = True

[[-1.-1.2246468e-16j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j -1.+1.2246468e-16j]]
u, v = (63619386817397248 + 104934693802365296ω + -272726231385265739ω² + 207873024016341045ω³), (-148110770243294308 + 259560599553875593ω + -180329611864894780ω² + 19912339200298870ω³)
u, v = (64096772492288733 + 103979922452582326ω + -271953805137549009ω² + 207690678914274805ω³), (-143503407527744767 + 113776229178869737ω + 48099584958109778ω² + -118417161702448815ω³)


KeyboardInterrupt: 

In [18]:
output = "1111112222222211111111221122221122112222221122222222111111112222222211112211112211111122221122222222111111112222222211111122221122112222222211111111222222221111111122111111222222221111111122222222111111112211221111111122222222111111112211111111222222221111112222222211111111222222221111111122112222222211111111"
print(Braid.evaluate_braid_string(output))
print(Braid.evaluate_individual_winding(output))

[[-1.00000000e+00-3.97036784e-07j  9.78967310e-07+1.98919372e-07j]
 [-9.78967312e-07+1.98919372e-07j -1.00000000e+00+3.97036854e-07j]]
(0, 0)
